In [18]:
from pathlib import Path
import geopandas as gpd
import pandas as pd
from sqlalchemy import text
from nvi_etl import make_engine_for

In [19]:
index_values = pd.read_csv("nvi_values_20250604.csv")
nvi_engine = make_engine_for("nvi_test")
etl_engine = make_engine_for("data")

In [42]:
ind_q = text(" SELECT * FROM indicator;")
geo_q = text(" SELECT * FROM location;")


with nvi_engine.connect() as db:
    indicator_names = pd.read_sql(ind_q, db)
    geographies = gpd.read_postgis(geo_q, db, geom_col="geometry")


In [57]:
col_renames = {
    row["id"]: row["name"] for _, row in 
    indicator_names.iterrows()
}

pivoted = (
    index_values
    .query("survey_question_id.isna()")
    # .merge(indicator_names.rename(columns={"id": "indicator_id"}), on="indicator_id")
    .pivot(
        index="location_id",
        columns="indicator_id",
        values="percentage"
    )
    .rename(columns=col_renames)
    .merge(geographies[["id", "geometry"]], left_index=True, right_on="id")
)

gdf = gpd.GeoDataFrame(pivoted, geometry="geometry")

gdf.to_file("nvi_results_20250717.geojson")